## 问题

### 1.如何判断一个变量是数组类型?
```
var arr = []
arr instanceof Array // true
```

### 2.写一个原型链继承的例子?

### 3.描述new一个对象的过程?

- 传参.
- 然后this先指向这个空对象.
- 然后进行属性赋值.
- 最后返回this
- f这个对象就具备了构造函数的所有属性.

### 4.zepto源码中如何使用原型链?

## 知识点

### 构造函数

In [1]:
// 构造函数
function Foo(name,age){
    this.name= name;
    this.age=age;
    this.class='class-1'
    // 默认有 return this
}
var f = new Foo('zhangsan',20)
f

Foo { name: 'zhangsan', age: 20, class: 'class-1' }

- 构造函数的特点
    + 字母名称大写
    + this指的是new出来的对象


- new一个对象的过程
    + 传参.
    + 然后this先变成一个空对象.
    + 然后进行属性赋值.
    + 默认将this,return回来.
    + f这个对象就具备了构造函数的所有属性.

### 构造函数-扩展

- var a = {}
- var a= []

- function Foo(){...}

等价于

- var Foo = new Function({...});

### 原型规则和示例

- 1.所有的引用类型(数组,对象,函数),都具有对象特性,即可自由扩展属性(除了null)

In [14]:
var obj={}
obj.a = 100;
obj

{ a: 100 }

In [16]:
var arr=[]
arr.a = 100
arr

[ a: 100 ]

In [23]:
function fn(){}
fn.a=100
fn.prototype.constructor


{ [Function: fn] a: 100 }

In [26]:
var foo = new fn()
foo.__proto__

fn {}

- 2.所有的引用类型(数组,对象,函数),都有一个 `__proto__ 属性` (隐式原型),属性值是一个普通的对象.

In [31]:
console.log(obj.__proto__)
console.log(arr.__proto__)
console.log(fn.__proto__)

{}
[]
[Function]


- 3.所有的函数,都有一个`prototype属性(显式原型)`,属性值也是一个普通的对象.

In [33]:
fn.prototype

fn {}

- 4.所有的引用类型(数组,对象,函数),`__proto`属性值指向它的构造函数的`prototype`属性值.

In [39]:
console.log(obj.__proto__ === Object.prototype)
var f_1 = new fn()
console.log(f_1.__proto__ === fn.prototype)

true
true


- 5.想要得到一个对象的某个属性时,如果这个对象本身没有这个属性,那么就会去它的`__proto__`(隐式原型)中去寻找.
  + 也就是去它的构造函数的显式原型中去查找.

In [56]:
function Foo(name,age){
    this.name = name;
    this.age = age;
}
// 构造函数的显式原型添加属性
Foo.prototype.printName = function(){
    console.log(this.name,"构造函数的显式原型创建属性");
}
// 创建对象
var f = new Foo("zhangsan",23);
f.printAge = function(){
    console.log(this.age,"在对象上直接添加属性")
}
f.printName();
f.printAge();
console.log(f.__proto__);
f.__proto__ === Foo.prototype

// this指向f

zhangsan 构造函数的显式原型创建属性
23 '在对象上直接添加属性'
Foo { printName: [Function] }


true

## 如何获得一个对象自身的属性?

上面的f对象本身有`name,age`,以及我们给这个对象扩展的属性`printAge`.

但是我们利用这个对象的构造函数扩展了一个属性(也就是在这个对象的显示原型对象上添加了一个属性).

为了剔除这个属性.我们可以使用下面的方法.

In [61]:
var item
for(item in f){
    if(f.hasOwnProperty(item)){
        console.log(item)
    }
}

name
age
printAge


### 原型链

In [68]:
function Foo(name,age){
    this.name = name;
    this.age = age;
}
// 构造函数的显式原型添加属性
Foo.prototype.printName = function(){
    console.log(this.name,"构造函数的显式原型创建属性");
}
// 创建对象
var f = new Foo("zhangsan",23);
f.printAge = function(){
    console.log(this.age,"在对象上直接添加属性")
}
f.printName();
f.printAge();
f.toString(); // 这一行就构成了原型链
// 利用 f.__proto__.__proto__的方式去寻找这个属性.
// 也就是去f的构造函数的隐式原型中去寻找.
// 也就是Object的隐式原型中去寻找


zhangsan 构造函数的显式原型创建属性
23 '在对象上直接添加属性'


'[object Object]'

In [69]:
var n = new Number(2)
n.toString(); // toString()是顶级构造函数上具有的属性.

'2'

![](https://upload-images.jianshu.io/upload_images/7505161-d0e79a98965413cd.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

![](https://upload-images.jianshu.io/upload_images/7505161-6cf11df534bf1082.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)


### instanceof

我们使用`instanceof`来判断`引用类型`是哪个`构造函数`.

`instanceof运算符用于测试构造函数的prototype属性是否出现在对象的原型链中的任何位置`

In [72]:
var a =[]
a instanceof Array

true

他的判断逻辑是: 
 - a 是否是Array这个构造函数new出来的.
 - a的`__proto__`(隐式原型对象)一层层的往上,是否对应到`Foo.prototype`
 - 再试着判断`a instanceof Object`
     + a的`__proto__`一层一层的往上找,可以找到`Object.prototype`

### 原型链继承的例子

#### 封装一个DOM查询

- 获得DOM节点
- 替换html文档
- 绑定一个事件

```js
function Elem(id){
    this.elem = document.getElementById(id)
}

Elem.prototype.html = function(val){
    var elem = this.elem
    if(val){
        elem.innerHTML = val 
        return this // 链式操作
    }else{
        return elem.innerHTML;
    }
}
Elem.prototype.on = function(type,fn){
    var elem = this.elem
    elem.addEventListener(type,fn)
    return this
}
var div1 = new Elem('main-page')
div1.html("<p>DOM 封装</p>").on('click',()=>{
    alert('hi')
}).html('<p>javascript</p>')
```